In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [7]:
from fastai.text.all import *
!pip install trax
import trax
from trax import layers as tl
from trax.supervised import training
from trax.fastmath import numpy as fastnp
import random as rnd

rnd.seed(11)

     |████████████████████████████████| 521 kB 871 kB/s eta 0:00:01
     |████████████████████████████████| 46 kB 1.9 MB/s  eta 0:00:01
     |████████████████████████████████| 520 kB 7.2 MB/s eta 0:00:01
     |████████████████████████████████| 33.3 MB 17.4 MB/s eta 0:00:01     |██████████████████████████████▋ | 31.8 MB 17.4 MB/s eta 0:00:01
     |████████████████████████████████| 214 kB 56.3 MB/s eta 0:00:01
     |████████████████████████████████| 361 kB 55.1 MB/s eta 0:00:01
     |████████████████████████████████| 64 kB 2.1 MB/s  eta 0:00:01


     |████████████████████████████████| 3.4 MB 52.7 MB/s eta 0:00:01
     |████████████████████████████████| 394.7 MB 9.7 kB/s  eta 0:00:01    |████                            | 48.8 MB 43.2 MB/s eta 0:00:09     |███████▏                        | 87.8 MB 39.6 MB/s eta 0:00:08     |████████████████████▏           | 248.7 MB 31.9 MB/s eta 0:00:05     |███████████████████████▊        | 293.1 MB 46.8 MB/s eta 0:00:03
     |████████████████████████████████| 3.8 MB 48.5 MB/s eta 0:00:01
     |████████████████████████████████| 14.8 MB 53.4 MB/s eta 0:00:01
     |████████████████████████████████| 462 kB 42.0 MB/s eta 0:00:01


     |████████████████████            | 2.3 MB 46.7 MB/s eta 0:00:01

     |███████████████████████████████▉| 3.7 MB 46.7 MB/s eta 0:00:01

     |████████████████████████████████| 3.7 MB 46.7 MB/s eta 0:00:01
  Created wheel for jax: filename=jax-0.2.7-py3-none-any.whl size=606843 sha256=29cce3e00409619a573e1ae99315adc69775b5dc5d482b564fffc00f9ea935d5
  Stored in directory: /root/.cache/pip/wheels/97/50/53/3c741874499adfb340bdb444dee5878c432673318c2df1dc86
Successfully built jax
  Attempting uninstall: six
    Found existing installation: six 1.14.0
    Uninstalling six-1.14.0:
      Successfully uninstalled six-1.14.0
  Attempting uninstall: wheel
    Found existing installation: wheel 0.34.2
    Uninstalling wheel-0.34.2:
      Successfully uninstalled wheel-0.34.2
  Attempting uninstall: numpy
    Found existing installation: numpy 1.18.5
    Uninstalling numpy-1.18.5:
      Successfully uninstalled numpy-1.18.5
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.34.0
    Uninstalling grpcio-1.34.0:
      Successfully uninstalled grpcio-1.34.0
  Attempting uninstall: tensorflow-estimator
    Found e

In [2]:
!unzip '../input/quora-question-pairs/train.csv.zip'

Archive:  ../input/quora-question-pairs/train.csv.zip
  inflating: train.csv               


In [4]:
df = pd.read_csv('./train.csv',low_memory=False)
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
len(df)

404290

In [13]:
train,val = RandomSplitter(valid_pct=0.05)(df)

In [19]:
len(train),len(val)

(384076, 20214)

In [20]:
train = df.loc[train,:][df.loc[train,:].is_duplicate==1]
val = df.loc[val,:]

In [21]:
train = pd.DataFrame(pd.concat([train.question1,train.question2]),columns=['qs'])
train.head()

,qs
232851,What is the quickest and less painful way to commit suicide?
198053,How can I meet PM modi?
377144,How do I earn money in garments online?
153782,What is some free editing video software for YouTube videos?
224115,How do I start learning C from basics to an expert level?


In [22]:
len(train)

283648

In [23]:
df_train,count = tokenize_df(train,text_cols=0)

In [24]:
df_train.head()

,text,text_length
232851,"[xxbos, xxmaj, what, is, the, quickest, and, less, painful, way, to, commit, suicide, ?]",14
198053,"[xxbos, xxmaj, how, can, i, meet, xxup, pm, modi, ?]",10
377144,"[xxbos, xxmaj, how, do, i, earn, money, in, garments, online, ?]",11
153782,"[xxbos, xxmaj, what, is, some, free, editing, video, software, for, youtube, videos, ?]",13
224115,"[xxbos, xxmaj, how, do, i, start, learning, c, from, basics, to, an, expert, level, ?]",15


In [25]:
half = len(df_train)//2
half

141824

In [26]:
df_train = pd.DataFrame({'question1':df_train['text'][:half],
                         'question2':df_train['text'][half:]})
df_train.head()

,question1,question2
232851,"[xxbos, xxmaj, what, is, the, quickest, and, less, painful, way, to, commit, suicide, ?]","[xxbos, xxmaj, what, can, be, the, most, insane, ways, to, commit, suicide, ?]"
198053,"[xxbos, xxmaj, how, can, i, meet, xxup, pm, modi, ?]","[xxbos, xxmaj, narendra, xxmaj, modi, :, xxmaj, how, should, i, meet, xxmaj, modi, ?]"
377144,"[xxbos, xxmaj, how, do, i, earn, money, in, garments, online, ?]","[xxbos, xxmaj, what, is, the, best, way, to, earn, money, in, garments, business, online, ?]"
153782,"[xxbos, xxmaj, what, is, some, free, editing, video, software, for, youtube, videos, ?]","[xxbos, xxmaj, what, are, some, best, free, xxmaj, video, editing, softwares, for, youtube, videos, ?]"
224115,"[xxbos, xxmaj, how, do, i, start, learning, c, from, basics, to, an, expert, level, ?]","[xxbos, xxmaj, how, and, where, do, i, start, learning, c, programming, ?]"


In [27]:
df_train.reset_index(inplace=True)

In [28]:
vocab = list(count.keys())

In [29]:
num = Numericalize(vocab=vocab)

In [30]:
df_train.question1 = df_train.question1.map(num).map(np.array).map(list)

df_train.question2 = df_train.question2.map(num).map(np.array).map(list)

In [31]:
df_train.head()

,index,question1,question2
0,232851,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]","[0, 1, 2, 15, 417, 4, 45, 1011, 108, 10, 11, 12, 13]"
1,198053,"[0, 1, 14, 15, 16, 17, 18, 19, 20, 13]","[0, 1, 717, 1, 20, 263, 1, 14, 187, 16, 17, 1, 20, 13]"
2,377144,"[0, 1, 14, 21, 16, 22, 23, 24, 25, 26, 13]","[0, 1, 2, 3, 4, 107, 9, 10, 22, 23, 24, 25, 935, 26, 13]"
3,153782,"[0, 1, 2, 3, 27, 28, 29, 30, 31, 32, 33, 34, 13]","[0, 1, 2, 50, 27, 107, 28, 1, 30, 29, 6530, 32, 33, 34, 13]"
4,224115,"[0, 1, 14, 21, 16, 35, 36, 37, 38, 39, 10, 40, 41, 42, 13]","[0, 1, 14, 6, 84, 21, 16, 35, 36, 37, 154, 13]"


In [32]:
df_test = pd.DataFrame(pd.concat([val.question1,val.question2],names='qs'))
df_test.head()

,0
9171,What is one of life's simplest pleasures?
178011,How do I cancel Quora?
213471,What are the differences between public sector and govt sector?
226276,Do mermaids and vampires exist?
62095,What can hamsters eat besides hamster food?


In [33]:
len(df_test)

40428

In [34]:
df_test,_ = tokenize_df(df_test,text_cols=0)

In [35]:
half_test = len(df_test)//2;half_test

20214

In [36]:
df_test = pd.DataFrame({'question1':df_test.text[:half_test],'question2':df_test.text[half_test:],'is_duplicate':val.is_duplicate})
df_test.head()

,question1,question2,is_duplicate
9171,"[xxbos, xxmaj, what, is, one, of, life, 's, simplest, pleasures, ?]","[xxbos, xxmaj, what, are, life, 's, simple, pleasures, ?]",1
178011,"[xxbos, xxmaj, how, do, i, cancel, xxmaj, quora, ?]","[xxbos, xxmaj, how, do, i, withdraw, at, xxmaj, quora, ?]",1
213471,"[xxbos, xxmaj, what, are, the, differences, between, public, sector, and, govt, sector, ?]","[xxbos, xxmaj, what, is, difference, between, the, private, and, public, sector, ?]",0
226276,"[xxbos, xxmaj, do, mermaids, and, vampires, exist, ?]","[xxbos, xxmaj, did, vampires, really, existed, ?]",1
62095,"[xxbos, xxmaj, what, can, hamsters, eat, besides, hamster, food, ?]","[xxbos, xxmaj, can, hamsters, eat, carrot, ?, xxmaj, why, or, why, not, ?]",1


In [37]:
df_test.reset_index(inplace=True)

In [38]:
df_test.question1 = df_test.question1.map(num).map(np.array).map(list)

df_test.question2 = df_test.question2.map(num).map(np.array).map(list)

In [39]:
df_test.head()

,index,question1,question2,is_duplicate
0,9171,"[0, 1, 2, 3, 413, 57, 174, 213, 7126, 21368, 13]","[0, 1, 2, 50, 174, 213, 547, 21368, 13]",1
1,178011,"[0, 1, 14, 21, 16, 2483, 1, 133, 13]","[0, 1, 14, 21, 16, 2249, 127, 1, 133, 13]",1
2,213471,"[0, 1, 2, 50, 4, 1650, 334, 751, 3803, 6, 3415, 3803, 13]","[0, 1, 2, 3, 521, 334, 4, 579, 6, 751, 3803, 13]",0
3,226276,"[0, 1, 21, 5252, 6, 20466, 291, 13]","[0, 1, 209, 20466, 101, 4917, 13]",1
4,62095,"[0, 1, 2, 15, 2966, 1040, 2967, 2968, 1671, 13]","[0, 1, 15, 2966, 1040, 16589, 13, 1, 232, 338, 232, 309, 13]",1


In [40]:
' '.join(num.vocab[i] for i in df_test.question1[0])

"xxbos xxmaj what is one of life 's simplest pleasures ?"

In [41]:
cut = int(len(df_train)*0.8)
train_Q1,train_Q2 = df_train.question1[:cut].tolist(),df_train.question2[:cut].tolist()
val_Q1,val_Q2 = df_train.question1[cut:].tolist(),df_train.question2[cut:].tolist()

In [42]:
num.o2i['xxpad']=-1

In [43]:
def data_generator(Q1, Q2, batch_size, pad=-1, shuffle=True):

    input1 = []
    input2 = []
    idx = 0
    len_q = len(Q1)
    question_indexes = [*range(len_q)]
    
    if shuffle:
        rnd.shuffle(question_indexes)
    
    while True:
        if idx >= len_q:
            idx = 0
            if shuffle:
                rnd.shuffle(question_indexes)
#         print(len_q,idx)
        q1 = Q1[question_indexes[idx]]
        q2 = Q2[question_indexes[idx]]
        
        idx += 1
        
        input1.append(q1)
        input2.append(q2)
        
        if len(input1) == batch_size:
            max_len = len(max(max(input1,key=len),max(input2,key=len),key=len))
            max_len =  2**int(np.ceil(np.log2(max_len)))
#             print(max_len)
            b1 = []
            b2 = []
            for q1, q2 in zip(input1, input2):  
#                 print(q1.shape,q2.shape)
                q1 = q1+[pad]*(max_len-len(q1))                
                q2 = q2+[pad]*(max_len-len(q2))
                
                b1.append(q1)                
                b2.append(q2)
            yield np.array(b1), np.array(b2)

            input1, input2 = [], [] 

In [44]:
batch_size = 2
res1, res2 = next(data_generator(train_Q1, train_Q2, batch_size))
print("First questions  : ",'\n', res1, '\n')
print("Second questions : ",'\n', res2)

First questions  :  
 [[   0    1    2   15   16   21 1247  382   10 1250    4  767   57    1
  6137   13]
 [   0    1   14   21   16  820  935 6849   13   -1   -1   -1   -1   -1
    -1   -1]] 

Second questions :  
 [[   0    1    2   15  188   21   32    4  767   57    1 6137   13   -1
    -1   -1]
 [   0    1   14   15   16  199   74  935 6849   13   -1   -1   -1   -1
    -1   -1]]


In [45]:
def Siamese(vocab_size=len(vocab)+1, d_model=128, mode='train'):

    def normalize(x):  # normalizes the vectors to have L2 norm 1
        return x / fastnp.sqrt(fastnp.sum(x * x, axis=-1, keepdims=True))
    
    q_processor = tl.Serial(  # Processor will run on Q1 and Q2.
        tl.Embedding(vocab_size,d_model), # Embedding layer
        tl.LSTM(d_model), # LSTM layer
        tl.Mean(axis=1), # Mean over columns
        tl.Fn('Normalize', lambda x: normalize(x))  # Apply normalize function
    )  # Returns one vector of shape [batch_size, d_model].
        
    # Run on Q1 and Q2 in parallel.
    model = tl.Parallel(q_processor, q_processor)
    return model

In [46]:
model = Siamese();model

Parallel_in2_out2[
  Serial[
    Embedding_30191_128
    LSTM_128
    Mean
    Normalize
  ]
  Serial[
    Embedding_30191_128
    LSTM_128
    Mean
    Normalize
  ]
]

In [47]:
def TripletLossFn(v1, v2, margin=0.25):
    """Custom Loss function.

    Args:
        v1 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q1.
        v2 (numpy.ndarray): Array with dimension (batch_size, model_dimension) associated to Q2.
        margin (float, optional): Desired margin. Defaults to 0.25.

    Returns:
        jax.interpreters.xla.DeviceArray: Triplet Loss.
    """
    
    scores = fastnp.dot(v1,v2.T) 

    batch_size = len(scores)
    # use fastnp to grab all postive `diagonal` entries in `scores`
    positive = fastnp.diagonal(scores)  # the positive ones (duplicates)

    # multiply `fastnp.eye(batch_size)` with 2.0 and subtract it out of `scores`
    negative_without_positive = scores-fastnp.eye(batch_size)*2

    # take the row by row `max` of `negative_without_positive`. 
    closest_negative = negative_without_positive.max(axis=[1])

    # subtract `fastnp.eye(batch_size)` out of 1.0 and do element-wise multiplication with `scores`
    negative_zero_on_duplicate = (1-fastnp.eye(batch_size))*scores
    
    # use `fastnp.sum` on `negative_zero_on_duplicate` for `axis=1` and divide it by `(batch_size - 1)` 
    mean_negative = fastnp.sum(negative_zero_on_duplicate,axis=1)/(batch_size-1)
    # compute `fastnp.maximum` among 0.0 and `A`
    # A = subtract `positive` from `margin` and add `closest_negative` 
    triplet_loss1 = fastnp.maximum(margin-positive+closest_negative,0.0)
    
    # compute `fastnp.maximum` among 0.0 and `B`
    # B = subtract `positive` from `margin` and add `mean_negative`
    triplet_loss2 = fastnp.maximum(margin-positive+mean_negative,0)

    # add the two losses together and take the `fastnp.mean` of it
    triplet_loss = fastnp.mean(triplet_loss1+triplet_loss2)
#     print(triplet_loss)
    

    
    return triplet_loss

In [48]:
from functools import partial
def TripletLoss(margin=0.25):
    triplet_loss_fn = partial(TripletLossFn, margin=margin)
    return tl.Fn('TripletLoss', triplet_loss_fn)

In [61]:
batch_size = 256
train_generator = data_generator(train_Q1, train_Q2, batch_size)
val_generator = data_generator(val_Q1, val_Q2, batch_size)
# print('train_Q1.shape ', train_Q1.shape)
# print('val_Q1.shape   ', val_Q1.shape)

In [62]:
lr_schedule = trax.lr.warmup_and_rsqrt_decay(400, 0.01)

In [67]:
def train_model(Siamese, TripletLoss, lr_schedule, train_generator=train_generator, val_generator=val_generator, output_dir='model/'):
    """Training the Siamese Model

    Args:
        Siamese (function): Function that returns the Siamese model.
        TripletLoss (function): Function that defines the TripletLoss loss function.
        lr_schedule (function): Trax multifactor schedule function.
        train_generator (generator, optional): Training generator. Defaults to train_generator.
        val_generator (generator, optional): Validation generator. Defaults to val_generator.
        output_dir (str, optional): Path to save model to. Defaults to 'model/'.

    Returns:
        trax.supervised.training.Loop: Training loop for the model.
    """
    output_dir = os.path.expanduser(output_dir)

    train_task = training.TrainTask(
        labeled_data=train_generator,       
        loss_layer=TripletLoss(),         
        optimizer=trax.optimizers.Adam(0.01),
        lr_schedule=lr_schedule, 
    )

    eval_task = training.EvalTask(
        labeled_data=val_generator,      
        metrics=[TripletLoss()],          
    )
    
    training_loop = training.Loop(Siamese(),
                                  train_task,
                                  eval_tasks=eval_task,
                                  output_dir=output_dir)

    return training_loop

In [68]:
train_steps = 100
training_loop = train_model(Siamese, TripletLoss, lr_schedule)
training_loop.run(n_steps = train_steps)


Step    100: Ran 99 train steps in 70.57 secs
Step    100: train TripletLoss |  0.49994019
Step    100: eval  TripletLoss |  0.49965480


In [57]:
def classify(test_Q1, test_Q2, y, threshold, model, vocab, data_generator=data_generator, batch_size=64):
    """Function to test the accuracy of the model.

    Args:
        test_Q1 (numpy.ndarray): Array of Q1 questions.
        test_Q2 (numpy.ndarray): Array of Q2 questions.
        y (numpy.ndarray): Array of actual target.
        threshold (float): Desired threshold.
        model (trax.layers.combinators.Parallel): The Siamese model.
        vocab (collections.defaultdict): The vocabulary used.
        data_generator (function): Data generator function. Defaults to data_generator.
        batch_size (int, optional): Size of the batches. Defaults to 64.

    Returns:
        float: Accuracy of the model.
    """
    accuracy = 0
    ### START CODE HERE (Replace instances of 'None' with your code) ###
    for i in range(0, len(test_Q1), batch_size):
        # Call the data generator (built in Ex 01) with shuffle=False using next()
        # use batch size chuncks of questions as Q1 & Q2 arguments of the data generator. e.g x[i:i + batch_size]
        # Hint: use `vocab['<PAD>']` for the `pad` argument of the data generator
        q1, q2 = next(data_generator(test_Q1[i:i+batch_size],test_Q2[i:i+batch_size],
                                     batch_size,shuffle=False))
        # use batch size chuncks of actual output targets (same syntax as example above)
        
        y_test = y[i:i+batch_size]
#         print(y_test.shape)
        # Call the model
        v1, v2 = model([q1,q2])
#         print(v1,v2)
#         print(i)
        for j in range(batch_size):
            # take dot product to compute cos similarity of each pair of entries, v1[j], v2[j]
            # don't forget to transpose the second argument
            d = fastnp.dot(v1[j],v2[j].T)
            # is d greater than the threshold?
            res = d>threshold
#             print(j,res)
            # increment accurancy if y_test is equal `res`
            accuracy += float(y_test[j]==res)
    # compute accuracy using accuracy and total length of test questions
    accuracy = accuracy/len(test_Q1)
    ### END CODE HERE ###
    
    return accuracy

In [54]:
Q1_test = df_test.question1.tolist()[:512*39]
Q2_test = df_test.question2.tolist()[:512*39]
y_test = df_test.is_duplicate.tolist()[:512*39]

In [69]:
model = Siamese()
model.init_from_file('./model/model.pkl.gz')

In [70]:
accuracy = classify(Q1_test[:10240],Q2_test[:10240], y_test[:10240], 0.7, model, vocab, batch_size = 512)
print("Accuracy", accuracy)

Accuracy 0.3681891025641026
